# Coursera Capstone Notebook

## Purpose

Classify Toronto neighbourhoods into 5 Clusters


## Data Requirements

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### Import libraries

In [21]:
#Copy from Lab
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


### Import Data

In [22]:
# create URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

wiki_page = requests.get(url)

### Parse HTML Wikimeadia page to extract the Table of postcodes

In [23]:
#Code from online question
from bs4 import BeautifulSoup 
soup = BeautifulSoup(wiki_page.text, 'html.parser')

table = soup.find('table', {'class': "wikitable sortable"})
tbody = table.find('tbody')
tr = tbody.findNext('tr')
html_tables = pd.read_html(url)

### Create Dataframes

In [29]:
toronto_pd = html_tables[0]
toronto_pd.columns = ['PostalCode','Borough','Neighborhood']
toronto_pd.drop(toronto_pd.index[0], inplace=True) #Drorp the title row
print(toronto_pd.shape)
toronto_pd.head()

(288, 3)


,PostalCode,Borough,Neighborhood
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


### Drop Boroughs if Not assigned

In [30]:
toronto_pd = toronto_pd[toronto_pd.Borough != 'Not assigned']
print(toronto_pd.shape)
toronto_pd.head()

(212, 3)


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


### Replace Neighbourhoods

In [33]:
toronto_pd.Neighbourhood = np.where(toronto_pd["Neighborhood"] == 'Not assigned', toronto_pd.Borough,toronto_pd.Neighborhood)
toronto_pd.head(10)


,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


### Join Boroughs

In [34]:
unique_toronto_pd = toronto_pd.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()
unique_toronto_pd.shape

(103, 3)